# Single AI crypto concept

In [ ]:
from xdata_config import *

## 1 - Imports and fonctions

In [ ]:
from functions_module import *

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.constraints import MaxNorm
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense 
from keras.models import load_model
from keras.layers import BatchNormalization, Dense

## Special list if needed

In [ ]:
Binance_USDT_HALAL.index("ROSE/USDT")

In [ ]:
# from datetime import date
# TICKERS = "../Binance-Fast-Trade-Bot/volatile_volume_" + str(date.today()) + ".txt"
# VOLATILE_COINS=[line.strip() for line in open(TICKERS)]
# PAIR_WITH="USDT"
# VOLATILE_USDT_PAIRS=[coin+"/USDT" for coin in VOLATILE_COINS]
# VOLATILE_BUSD_PAIRS=[coin+"/BUSD" for coin in VOLATILE_COINS]
# VOLATILE_USDT_PAIRS

# coins_to_download=''
# for coin in VOLATILE_COINS:
#     coins_to_download=coins_to_download+" "+coin
# f"node database/ddargs.js {coins_to_download} {PAIR_WITH}"

In [ ]:
# coins_to_download=''
# for coin in VOLATILE_COINS:
#     coins_to_download=coins_to_download+" "+coin
# os.system(f"node database/ddargs.js {coins_to_download} {PAIR_WITH}")#node database/ddargs.js ORN BUSD

In [ ]:
# pair_list = find_intersection(VOLATILE_USDT_PAIRS,Binance_USDT_HALAL)
# #tf = '1m'
# oldest_pair = "BTC/USDT"
# if oldest_pair not in pair_list: pair_list.append(oldest_pair)
# df_list1m = {}
# df_list1d = {}
# df_list1h = {}
# df_list5m = {}
# df_list15m = {}


# for pair in pair_list:
#     df = get_historical_from_db(ccxt.binance(), pair, '1m', path="./database/")
#     df_list1m[pair] = df.loc[:]

# for pair in pair_list:
#     df = get_historical_from_db(ccxt.binance(), pair, '1d', path="./database/")
#     df_list1d[pair] = df.loc[:]

# for pair in pair_list:
#     df = get_historical_from_db(ccxt.binance(), pair, '1h', path="./database/")
#     df_list1h[pair] = df.loc[:]

# for pair in pair_list:
#     df = get_historical_from_db(ccxt.binance(), pair, '5m', path="./database/")
#     df_list5m[pair] = df.loc[:]

# for pair in pair_list:
#     df = get_historical_from_db(
#         ccxt.binance(), pair, '15m', path="./database/")
#     df_list15m[pair] = df.loc[:]
# del(df)
# df_list = df_list1m
# prerr("Data load 100% use df_list1d[\"BTC/USDT\"] for exemple to access")



## Checking the import

In [ ]:
# chking import
MetaData

In [ ]:
if BUY_MODE=="BUY_ONLY":
    buy_function=buy_up_only
elif BUY_MODE=="BUY_UP":
    buy_function=buy_up
elif  BUY_MODE=="BUY_DIP":
    buy_function=buy_min_up
elif  BUY_MODE=="AFTER_DEPTH":
    buy_function=buy_after_depth
elif  BUY_MODE=="BUY_UP_CLOSE":
    buy_function=buy_up_close
elif  BUY_MODE=="AFTER_DEPTH_CLOSE":
    buy_function=buy_after_depth_close
elif  BUY_MODE=="BUY_TEST":
    buy_function=buy_test
elif BUY_MODE=="BUY_MIN_CLOSE":
    buy_function=buy_min_close
elif  BUY_MODE=="SELL_TEST":
    buy_function=sell_test
elif  BUY_MODE=="BUY_FIX":
    buy_function=buy_fix
elif  BUY_MODE=="BUY_OPTIMAL":
    buy_function=buy_optimal


try:
    os.mkdir(DATA_DIR, mode = 0o777)
except Exception as e:
    print(e)
print(f"Results dir: {DATA_DIR}")



In [ ]:
xdf=pd.DataFrame()
count=0
row_numbers=30000
for pair in pair_list:
    if pair != "BTC/USDT" and pair != "EUR/USDT" and pair != "ETH/USDT" :
        print("working on: "+pair ,end=" -->")
        try:
            df=mini_expand4(pair=pair,i=0,j=len(df_list1m[pair]),window=WINDOW_SIZE,metadata=MetaData,buy_pourcent=BUY_PERCENT,sell_pourcent=SELL_PERCENT,buy_function=buy_function)
            print("df original shape "+str(df.shape))
            print(f"df original shape buy mean : {df.buy.mean()*100}")
            df=df.reset_index()
            try:df.pop("num_index")
            except: pass
            try:df.pop("index")
            except: pass
            try:df.pop("date")
            except: pass
            df=data_shufler(df)            
            #df=data_chooser(df,weight=50,row_numbers=df.buy.sum()*2)
            df=data_chooser50(df,row_numbers=row_numbers)
            gc.collect()
            df=data_cleanup(df)
            df=df.dropna()
            print(pair+f" is processed -- {count}/{len(pair_list)}")
        except Exception as e:
            print(f"error while processing {pair} {count}/{len(pair_list)}")
            print(e)
        xdf=pd.concat([xdf,df],axis=0)
        count+=1
        del(df)
        gc.collect()
df=xdf
del xdf
gc.collect()


In [ ]:
df=df.reset_index().drop(columns="num_index")
gc.collect()
for i in range(1):
    df = df.reindex(np.random.permutation(df.index)).reset_index().drop(columns="index")
    gc.collect()
df

In [ ]:
df.to_feather(f"../Data/fea/w{WINDOW_SIZE}_buy{BUY_PERCENT}_forcasr{MAX_FORCAST_SIZE}min_{BUY_MODE}.fea")

In [ ]:
gc.collect()
print("df choosen data shape"+str(df.shape))
print(f"pair: {(df.shape[0]/2)==df.buy.sum()}")
dt=df.to_numpy(dtype=np.float32)
#dt=df.to_numpy()
dt=np.nan_to_num(dt,nan=0)
#dt=dt.astype(np.float32)
dt=np.nan_to_num(dt, neginf=0) 
dt=np.nan_to_num(dt, posinf=0) 

index_20percent= int(0.2*len(dt[:,0]))
print(index_20percent)


In [ ]:
# # feather loading
# df=pd.read_feather(f"../Data/fea/w{WINDOW_SIZE}_buy{BUY_PERCENT}_forcasr{MAX_FORCAST_SIZE}min_{BUY_MODE}.fea")
# dt=df.to_numpy(dtype=np.float32)
# dt=fixdt(dt)
# index_20percent= int(0.2*len(dt[:,0]))
# gc.collect()



## Self-Normalized Model

In [ ]:

## Normalzed Model
IN_DIM=dt.shape[1]-1
model = Sequential()
model.add(BatchNormalization(input_shape=(IN_DIM,)))
model.add(Dense(int(250),activation='relu')) 
model.add(BatchNormalization())
model.add(Dense(int(20),activation='relu')) 
model.add(Dense(int(50),activation='relu')) 
model.add(Dense(1,activation='sigmoid'))
print(model.summary())
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
callbacks_a = ModelCheckpoint(filepath =Model_FileName,monitor ='val_accuracy',save_best_only = True, save_weights = True)
callbacks_b = EarlyStopping(monitor ='val_accuracy',mode='auto',patience=15,verbose=1)
print("saving file in: "+Model_FileName)
history = model.fit(dt[index_20percent:, 0:-1],
                dt[index_20percent:,-1],
                validation_data=(dt[:index_20percent, :-1],dt[:index_20percent,-1]),
                epochs=6000,
                batch_size=256*10,
                callbacks=[callbacks_a,callbacks_b])

print('##########################################################################')
print(f"------val_accuracy-----> {'{0:.4g}'.format(max(history.history['val_accuracy'])*100)} | {'{0:.4g}'.format(max(history.history['accuracy'])*100)} <----------accuracy----------")
print(Normalization_File)
print(Model_FileName)
model_init_file=Model_FileName.replace(f"_v{VERSION}", "_vInit")
print(f"save to: {model_init_file}")
model.save(model_init_file)
model_init=model

In [ ]:
# model_init_file=Model_FileName.replace(f"_v{VERSION}", "_vInit")
# print(f"save to: {model_init_file}")
# model.save(model_init_file)
# model_init=model

## 2- Model Plus

In [ ]:
import gc
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import Nadam
from keras.callbacks import ModelCheckpoint, EarlyStopping
# Define the class weights
class_weights = {0: 1., 1: 1.}

gc.collect()

SizeTunner = 1
IN_DIM = dt.shape[1] - 1

model = Sequential()
model.add(BatchNormalization(input_shape=(IN_DIM,)))
model.add(Dense(int(300 * SizeTunner), activation='elu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(int(200 * SizeTunner), activation='elu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(int(80 * SizeTunner), activation='elu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(int(80 * SizeTunner), activation='elu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(int(80 * SizeTunner), activation='elu'))
model.add(BatchNormalization())
model.add(Dense(int(80 * SizeTunner), activation='elu'))
model.add(BatchNormalization())
model.add(Dense(int(80 * SizeTunner), activation='relu'))
model.add(BatchNormalization())
model.add(Dense(int(20 * SizeTunner), activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1, activation='sigmoid'))

print(model.summary())

optimizer = Nadam(lr=0.002, beta_1=0.9, beta_2=0.999)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

callbacks = [
    ModelCheckpoint(filepath=Model_FileName, monitor='val_accuracy', save_best_only=True, save_weights=True),
    EarlyStopping(monitor='val_accuracy', mode='auto', patience=6, verbose=1)
]

print("saving file in: " + Model_FileName)
history = model.fit(dt[index_20percent:, :-1],
                    dt[index_20percent:, -1],
                    validation_data=(dt[:index_20percent, :-1], dt[:index_20percent, -1]),
                    epochs=500,
                    batch_size=256*10,
                    callbacks=callbacks,
                    class_weight=class_weights)

verydeep_model_file=f'{DATA_DIR}/tp{int(BUY_PERCENT*100)}_w{WINDOW_SIZE}_max{MAX_FORCAST_SIZE}min_Model'+"_VeryDeep.h5"
model.save(verydeep_model_file)
print(verydeep_model_file)
very_deep_model=load_model(f'{DATA_DIR}/tp{int(BUY_PERCENT*100)}_w{WINDOW_SIZE}_max{MAX_FORCAST_SIZE}min_Model'+"_VeryDeep.h5")
#868/868 [==============================] - 30s 35ms/step - loss: 0.6078 - accuracy: 0.6664 - val_loss: 0.6107 - val_accuracy: 0.6639 >0.6646
#Results after 380 min
# Epoch 133/500
# 347/347 [==============================] - 138s 398ms/step - loss: 0.5867 - accuracy: 0.6842 - val_loss: 0.5839 - val_accuracy: 0.6863
# Epoch 134/500
# 347/347 [==============================] - 137s 395ms/step - loss: 0.5865 - accuracy: 0.6843 - val_loss: 0.5845 - val_accuracy: 0.6861
# Epoch 134: early stopping



# 3 - Binary Model

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from brevitas.tensor import QuantTensor
from brevitas.core.quant import binary_sign
from brevitas.core.quant import ternary_sign

# Define the class weights
class_weights = {0: 1., 1: 1.}

# Set up input layer
IN_DIM = dt.shape[1] - 1
inputs = Input(shape=(IN_DIM,))

# Define BNN layers
q_activation = binary_sign
q_kernel = ternary_sign
q_bias = ternary_sign

x = BatchNormalization()(inputs)
x = Dense(300, activation=q_activation, kernel_quantizer=q_kernel, bias_quantizer=q_bias)(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)

x = Dense(200, activation=q_activation, kernel_quantizer=q_kernel, bias_quantizer=q_bias)(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)

x = Dense(80, activation=q_activation, kernel_quantizer=q_kernel, bias_quantizer=q_bias)(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)

x = Dense(80, activation=q_activation, kernel_quantizer=q_kernel, bias_quantizer=q_bias)(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)

x = Dense(80, activation=q_activation, kernel_quantizer=q_kernel, bias_quantizer=q_bias)(x)
x = BatchNormalization()(x)

x = Dense(80, activation=q_activation, kernel_quantizer=q_kernel, bias_quantizer=q_bias)(x)
x = BatchNormalization()(x)

x = Dense(20, activation=q_activation, kernel_quantizer=q_kernel, bias_quantizer=q_bias)(x)
x = BatchNormalization()(x)

outputs = Dense(1, activation='sigmoid')(x)

# Define the model
model = Model(inputs=inputs, outputs=outputs)
print(model.summary())

# Compile the model
optimizer = Nadam(lr=0.002, beta_1=0.9, beta_2=0.999)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Define callbacks
callbacks = [
    ModelCheckpoint(filepath=Model_FileName, monitor='val_accuracy', save_best_only=True, save_weights=True),
    EarlyStopping(monitor='val_accuracy', mode='auto', patience=6, verbose=1)
]

# Train the model
history = model.fit(QuantTensor(dt[index_20percent:, :-1], bit_width=1),
                    QuantTensor(dt[index_20percent:, -1], bit_width=1),
                    validation_data=(QuantTensor(dt[:index_20percent, :-1], bit_width=1), QuantTensor(dt[:index_20percent, -1], bit_width=1)),
                    epochs=500,
                    batch_size=256*10,
                    callbacks=callbacks,
                    class_weight=class_weights)

# Save the model
binary_model_file=f'{DATA_DIR}/tp{int(BUY_PERCENT*100)}_w{WINDOW_SIZE}_max{MAX_FORCAST_SIZE}min_Model'+"_BINARY.h5"
model.save(binary_model_file)
print(binary_model_file)

# Load the model
binary_deep_model_file=tf.keras.models.load_model(binary_model_file)


# Saving the model

In [ ]:
# mini_range_start=0
# mini_range_stop=200000
# model.evaluate(dt[mini_range_start:mini_range_stop,:-1],dt[mini_range_start:mini_range_stop,-1])

## 3-  Test

### stats plus

In [ ]:
USED_MODEL=very_deep_model
#model_init=model
#USED_MODEL=model_init#load_model("/UltimeTradingBot/Data/BUY_UP_CLOSE/tp60_w6_max3min_Model_GoodVeryDeep.h5")
Prediction_Note=USED_MODEL.predict( dt[:, 0:-1])
prediction2=Prediction_Note.round()
hp(prediction2[:,0].mean())
PesemisticPrediction=(Prediction_Note[:,0]-0.49).round()
hp(PesemisticPrediction.mean())
Y=dt[:,-1].copy()
Pred01=prediction2[:,-1]
Original_Traget_Data=Y
Predicted_Data=Pred01

TruePred=((Original_Traget_Data==Predicted_Data)).copy()
ModelAccuracy=hp(TruePred.mean(),"ModelAccuracy")

TrueWinPred=((Predicted_Data==1) & (Original_Traget_Data==1) ).copy()
TrueWinPred_Mean=hp(TrueWinPred.mean(),"True Win Predictions Mean of all")

LossPred=((Predicted_Data==1) & (Original_Traget_Data==0) ).copy()
LossPred_Mean=hp(LossPred.mean(),"XXX Loss Buy Mean of all")

MissedDealPred=((Predicted_Data==0) & (Original_Traget_Data==1) ).copy()
MissedDeal_Mean=hp(MissedDealPred.mean(),"Missed good deal off all")

GoodZeroPred=((Predicted_Data==0) & (Original_Traget_Data==0) ).copy()
GoodZero_Mean=hp(GoodZeroPred.mean(),"Good Zero prediction Mean")

fiability=TrueWinPred_Mean + LossPred_Mean + MissedDeal_Mean + GoodZero_Mean
if( fiability == 100):print("good fiability")
else: print(f"check the fiability {fiability}")

winratio=TrueWinPred_Mean/(LossPred_Mean+TrueWinPred_Mean)
print(f"========= Win Ratio:{winratio*100} ====================")


## ANTI Retrain 

In [ ]:
# good_dt=dt[TruePred]
# good_dt

In [ ]:
#bad_dt=dt[ np.logical_not(TruePred)]
bad_dt=dt[Predicted_Data==1 ]


In [ ]:
#Anti prediction

BadONE=bad_dt[bad_dt[:,-1]==0]
TrueOne=bad_dt[bad_dt[:,-1]==1][:BadONE.shape[0]]
AntiPrediction_DT=np.concatenate((BadONE,TrueOne),axis=0)
np.random.shuffle(AntiPrediction_DT)

retrain_dt=AntiPrediction_DT
print(f"Dataset Size is : {retrain_dt.shape[0]}")
class_1_weight=hp(retrain_dt[:,-1].mean())/100

import gc
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import Nadam
from keras.callbacks import ModelCheckpoint, EarlyStopping
# Define the class weights
index_20percent=int(retrain_dt.shape[1]*0.2)

class_weights = {0: 1-class_1_weight, 1: class_1_weight}

gc.collect()

SizeTunner = 1
IN_DIM = retrain_dt.shape[1] - 1

model = Sequential()
model.add(BatchNormalization(input_shape=(IN_DIM,)))
model.add(Dense(int(200 * SizeTunner), activation='elu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(int(80 * SizeTunner), activation='elu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

# model.add(Dense(int(80 * SizeTunner), activation='elu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.2))

# model.add(Dense(int(80 * SizeTunner), activation='elu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.2))

model.add(Dense(int(80 * SizeTunner), activation='elu'))
model.add(BatchNormalization())
model.add(Dense(int(80 * SizeTunner), activation='elu'))
model.add(Dense(int(80 * SizeTunner), activation='relu'))
model.add(Dense(int(20 * SizeTunner), activation='relu'))
model.add(Dense(1, activation='sigmoid'))

print(model.summary())

optimizer = Nadam(lr=0.002, beta_1=0.9, beta_2=0.999)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

callbacks = [
    ModelCheckpoint(filepath=Model_FileName, monitor='loss', save_best_only=True, save_weights=True),
    EarlyStopping(monitor='loss', mode='auto', patience=20, verbose=1)
]

print("saving file in: " + Model_FileName)
history = model.fit(retrain_dt[index_20percent:, :-1],
                    retrain_dt[index_20percent:, -1],
                    validation_data=(retrain_dt[:index_20percent, :-1], retrain_dt[:index_20percent, -1]),
                    epochs=500,
                    batch_size=256*5,
                    callbacks=callbacks,
                    class_weight=class_weights)

#868/868 [==============================] - 30s 35ms/step - loss: 0.6078 - accuracy: 0.6664 - val_loss: 0.6107 - val_accuracy: 0.6639 >0.6646
#Results after 380 min
# Epoch 133/500
# 347/347 [==============================] - 138s 398ms/step - loss: 0.5867 - accuracy: 0.6842 - val_loss: 0.5839 - val_accuracy: 0.6863
# Epoch 134/500
# 347/347 [==============================] - 137s 395ms/step - loss: 0.5865 - accuracy: 0.6843 - val_loss: 0.5845 - val_accuracy: 0.6861
# Epoch 134: early stopping
justgood_good_model=model
justgood_good_model_wheretosave=f'{DATA_DIR}/tp{int(BUY_PERCENT*100)}_w{WINDOW_SIZE}_max{MAX_FORCAST_SIZE}min_Anti-Model_v2.h5'
justgood_good_model.save(justgood_good_model_wheretosave)
print(justgood_good_model_wheretosave)

# True PredONly

In [23]:
#Change retaindt
for rrr in range(1,7):
    retrain_dt=dt
    class_1_weight=TrueWinPred.mean()

    import gc
    from keras.layers import Dense
    from keras.models import Sequential
    from keras.optimizers import Nadam
    from keras.callbacks import ModelCheckpoint, EarlyStopping
    # Define the class weights
    index_20percent=int(retrain_dt.shape[1]*0.2)
    class_weights = {0: 1-class_1_weight, 1: class_1_weight}
    gc.collect()

    SizeTunner = 1
    IN_DIM = retrain_dt.shape[1] - 1

    model = Sequential()
    model.add(BatchNormalization(input_shape=(IN_DIM,)))
    model.add(Dense(int(200 * SizeTunner), activation='elu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(Dense(int(80 * SizeTunner), activation='elu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    # model.add(Dense(int(80 * SizeTunner), activation='elu'))
    # model.add(BatchNormalization())
    # model.add(Dropout(0.2))

    # model.add(Dense(int(80 * SizeTunner), activation='elu'))
    # model.add(BatchNormalization())
    # model.add(Dropout(0.2))

    model.add(Dense(int(80 * SizeTunner), activation='elu'))
    model.add(BatchNormalization())
    model.add(Dense(int(80 * SizeTunner), activation='elu'))
    model.add(Dense(int(80 * SizeTunner), activation='relu'))
    model.add(Dense(int(20 * SizeTunner), activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    print(model.summary())

    optimizer = Nadam(lr=0.002, beta_1=0.9, beta_2=0.999)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    callbacks = [
        ModelCheckpoint(filepath=Model_FileName, monitor='val_accuracy', save_best_only=True, save_weights=True),
        EarlyStopping(monitor='val_accuracy', mode='auto', patience=16, verbose=1)
    ]

    print("saving file in: " + Model_FileName)
    history = model.fit(retrain_dt[index_20percent:, :-1],
                        TrueWinPred[index_20percent:],
                        validation_data=(retrain_dt[:index_20percent, :-1], TrueWinPred[:index_20percent]),
                        epochs=500,
                        batch_size=256*5,
                        callbacks=callbacks,
                        class_weight=class_weights)

    #868/868 [==============================] - 30s 35ms/step - loss: 0.6078 - accuracy: 0.6664 - val_loss: 0.6107 - val_accuracy: 0.6639 >0.6646
    #Results after 380 min
    # Epoch 133/500
    # 347/347 [==============================] - 138s 398ms/step - loss: 0.5867 - accuracy: 0.6842 - val_loss: 0.5839 - val_accuracy: 0.6863
    # Epoch 134/500
    # 347/347 [==============================] - 137s 395ms/step - loss: 0.5865 - accuracy: 0.6843 - val_loss: 0.5845 - val_accuracy: 0.6861
    # Epoch 134: early stopping

    true_win_model=model
    wheretosave=f'{DATA_DIR}/tp{int(BUY_PERCENT*100)}_w{WINDOW_SIZE}_max{MAX_FORCAST_SIZE}min_Model'+f"_true_win_model_Re{rrr}.h5"
    true_win_model.save(wheretosave)
    print(wheretosave)
    USED_MODEL=true_win_model
    bad_Prediction_Note=USED_MODEL.predict( dt[:, 0:-1])
    Pred02=bad_Prediction_Note.round()
    Original_Traget_Data=Y
    Predicted_Data=Pred02[:,0]

    BadTruePred=(Original_Traget_Data==Predicted_Data).copy()
    BadModelAccuracy=hp(BadTruePred.mean(),"ModelAccuracy")

    BadTrueWinPred=((Predicted_Data==1) & (Original_Traget_Data==1) ).copy()
    BadTrueWinPred_Mean=hp(BadTrueWinPred.mean(),"True Win Predictions Mean of all")

    BadLossPred=((Predicted_Data==1) & (Original_Traget_Data==0) ).copy()
    BadLossPred_Mean=hp(BadLossPred.mean(),"XXX Loss Buy Mean of all")

    BadMissedDealPred=((Predicted_Data==0) & (Original_Traget_Data==1) ).copy()
    BadMissedDeal_Mean=hp(BadMissedDealPred.mean(),"Missed good deal off all")

    BadGoodZeroPred=((Predicted_Data==0) & (Original_Traget_Data==0) ).copy()
    BadGoodZero_Mean=hp(BadGoodZeroPred.mean(),"Good Zero prediction Mean")

    fiability=BadTrueWinPred_Mean + BadLossPred_Mean + BadMissedDeal_Mean + BadGoodZero_Mean
    if( fiability == 100):print("good fiability")
    else: print(f"check the fiability {fiability}")
    winratio=BadTrueWinPred_Mean/(BadLossPred_Mean+BadTrueWinPred_Mean)
    print(f"========= Win Ratio:{winratio*100} ====================")
    ## for retraining again
    TrueWinPred=BadTrueWinPred

2555/2555 [==============================] - 154s 60ms/step - loss: 0.0881 - accuracy: 0.8749 - val_loss: 0.2367 - val_accuracy: 0.8761
Epoch 11/500
 929/2555 [=========>....................] - ETA: 1:51 - loss: 0.0876 - accuracy: 0.8757

In [ ]:
very_deep_good_model=model
wheretosave=f'{DATA_DIR}/tp{int(BUY_PERCENT*100)}_w{WINDOW_SIZE}_max{MAX_FORCAST_SIZE}min_Model'+"_GoodVeryDeep_v2.h5"
very_deep_good_model.save(wheretosave)
print(wheretosave)

In [ ]:
very_deep_bad_model=model
very_deep_bad_model.save(f'{DATA_DIR}/tp{int(BUY_PERCENT*100)}_w{WINDOW_SIZE}_max{MAX_FORCAST_SIZE}min_Model'+"_BadVeryDeep_v2.h5")

## Test On special coin

In [ ]:
## Generate Data
BAD_PERIOD_START="2022-08-30"
BAD_PERIOD_END="2022-11-22"
pair_to_test="GMT/USDT"
MAX_FORCAST_SIZE=5
USED_MODEL=very_deep_good_model#true_win_model#model_init #model_good_x3 #very_deep_good_model 16/1.7


loc_start=0
loc_end=1000000


i_start=71000
i_end=i_start+200

# loc_start=df_list1m[pair_to_test].index.get_loc(pd.to_datetime(BAD_PERIOD_START))
# loc_end=df_list1m[pair_to_test].index.get_loc(pd.to_datetime(BAD_PERIOD_END))


OnePair_DF=mini_expand4(        pair=pair_to_test,
                                i=loc_start,j=loc_end,
                                window=WINDOW_SIZE,
                                metadata=MetaData,
                                high_weight=1,
                                buy_pourcent=0.55,
                                sell_pourcent=SELL_PERCENT,
                                buy_function=buy_fix#buy_test
                        )
OnePair_DT=OnePair_DF.to_numpy()
gc.collect()
OnePair_DT=fixdt(OnePair_DT)
print(OnePair_DT[0,0] == OnePair_DF.iloc[0,0])
print(OnePair_DT[5,5] == OnePair_DF.iloc[5,5])
hp(OnePair_DF.buy.mean(),"Buy mean percent")

import matplotlib.pyplot as plt

x = np.linspace(0, 10, 500)
dashes = [10, 5, 100, 5]  # 10 points on, 5 off, 100 on, 5 off

fig, ax = plt.subplots()
line1, = ax.plot(OnePair_DF.index[i_start:i_end], OnePair_DF.price[i_start:i_end], '-', linewidth=1,
                 label='Dashes set retroactively')
line1.set_dashes(dashes)
plt.plot(OnePair_DF[i_start:i_end][OnePair_DF.buy[i_start:i_end]==1].index, OnePair_DF[i_start:i_end][OnePair_DF.buy[i_start:i_end]==1].price, 'ro')


ax.legend(loc='lower right')
plt.show()



OnePair_PredNote=USED_MODEL.predict( OnePair_DT[:, 0:-1])
OnePair_Pred=OnePair_PredNote.round()

gc.collect()

Original_Traget_Data=OnePair_DT[:,-1]
Predicted_Data=OnePair_Pred[:,0]
gc.collect()
TruePred=(Original_Traget_Data==Predicted_Data).copy()
ModelAccuracy=hp(TruePred.mean(),"ModelAccuracy")
gc.collect()
TrueWinPred=((Predicted_Data==1) & (Original_Traget_Data==1) ).copy()
TrueWinPred_Mean=hp(TrueWinPred.mean(),"True Win Predictions Mean of all")
gc.collect()
LossPred=((Predicted_Data==1) & (Original_Traget_Data==0) ).copy()
LossPred_Mean=hp(LossPred.mean(),"XXX Loss Buy Mean of all")
gc.collect()

MissedDealPred=((Predicted_Data==0) & (Original_Traget_Data==1) ).copy()
MissedDeal_Mean=hp(MissedDealPred.mean(),"Missed good deal off all")
gc.collect()

GoodZeroPred=((Predicted_Data==0) & (Original_Traget_Data==0) ).copy()
GoodZero_Mean=hp(GoodZeroPred.mean(),"Good Zero prediction Mean")
gc.collect()

fiability=TrueWinPred_Mean + LossPred_Mean + MissedDeal_Mean + GoodZero_Mean
if( fiability == 100):print("good fiability")
else: print(f"check the fiability {fiability}")
winratio=TrueWinPred_Mean/(LossPred_Mean+TrueWinPred_Mean)

print(f"========= Win Ratio:{winratio*100} %====================")




PREDICTION_TO_TEST=Predicted_Data

x = np.linspace(0, 10, 500)
dashes = [10, 5, 100, 5]  # 10 points on, 5 off, 100 on, 5 off
fig, ax = plt.subplots()
line1, = ax.plot(OnePair_DF.index[i_start:i_end], OnePair_DF.price[i_start:i_end], '-', linewidth=1,
                 label='price')
line1.set_dashes(dashes)
plt.plot(OnePair_DF[i_start:i_end][PREDICTION_TO_TEST[i_start:i_end]==1].index, OnePair_DF[i_start:i_end][PREDICTION_TO_TEST[i_start:i_end]==1].price, 'ro')


ax.legend(loc='lower right')
plt.show()


In [ ]:
XX=OnePair_DT[:100000,:-1]
YY=OnePair_DT[:100000,-1]
precision=0.0
# Good_Prediction_Note=very_deep_good_model.predict( XX)
# Bad_Prediction_Note=very_deep_bad_model.predict( XX)
# Initial_Pred_Note=model_init.predict( XX)
Predicted_Data=OnePair_Pred[:300000,0]
goodp=(Good_Prediction_Note-precision).round()
# badp=(Bad_Prediction_Note).round()
# initp=Initial_Pred_Note.round()

# Original_Traget_Data=YY

#Predicted_Data=((goodp==badp|initp==goodp))[:,0]
Predicted_Data=(goodp)[:,0]

GoodTruePred=(Original_Traget_Data==Predicted_Data).copy()
GoodModelAccuracy=hp(GoodTruePred.mean(),"ModelAccuracy")

GoodTrueWinPred=((Predicted_Data==1) & (Original_Traget_Data==1) ).copy()
GoodTrueWinPred_Mean=hp(GoodTrueWinPred.mean(),"True Win Predictions Mean of all")

GoodLossPred=((Predicted_Data==1) & (Original_Traget_Data==0) ).copy()
GoodLossPred_Mean=hp(GoodLossPred.mean(),"XXX Loss Buy Mean of all")

GoodMissedDealPred=((Predicted_Data==0) & (Original_Traget_Data==1) ).copy()
GoodMissedDeal_Mean=hp(GoodMissedDealPred.mean(),"Missed good deal off all")

GoodGoodZeroPred=((Predicted_Data==0) & (Original_Traget_Data==0) ).copy()
GoodGoodZero_Mean=hp(GoodGoodZeroPred.mean(),"Good Zero prediction Mean")

winratio=GoodTrueWinPred_Mean/(GoodTrueWinPred_Mean+GoodLossPred_Mean)
fiability=GoodTrueWinPred_Mean + GoodLossPred_Mean + GoodMissedDeal_Mean + GoodGoodZero_Mean
if( fiability == 100):print("good fiability")
else: print(f"check the fiability {fiability}")
print(f"========= Win Ratio:{winratio*100} ====================")


In [ ]:
def test_prediction(XX,YY,USEDMODEL)
XX=OnePair_DT[:100000,:-1]
YY=OnePair_DT[:100000,-1]

# Good_Prediction_Note=very_deep_good_model.predict( XX)
# Bad_Prediction_Note=very_deep_bad_model.predict( XX)
# Initial_Pred_Note=model_init.predict( XX)
goodp=(Good_Prediction_Note-precision).round()
# badp=(Bad_Prediction_Note).round()
# initp=Initial_Pred_Note.round()

# Original_Traget_Data=YY

#Predicted_Data=((goodp==badp|initp==goodp))[:,0]
Predicted_Data=(goodp)[:,0]

GoodTruePred=(Original_Traget_Data==Predicted_Data).copy()
GoodModelAccuracy=hp(GoodTruePred.mean(),"ModelAccuracy")

GoodTrueWinPred=((Predicted_Data==1) & (Original_Traget_Data==1) ).copy()
GoodTrueWinPred_Mean=hp(GoodTrueWinPred.mean(),"True Win Predictions Mean of all")

GoodLossPred=((Predicted_Data==1) & (Original_Traget_Data==0) ).copy()
GoodLossPred_Mean=hp(GoodLossPred.mean(),"XXX Loss Buy Mean of all")

GoodMissedDealPred=((Predicted_Data==0) & (Original_Traget_Data==1) ).copy()
GoodMissedDeal_Mean=hp(GoodMissedDealPred.mean(),"Missed good deal off all")

GoodGoodZeroPred=((Predicted_Data==0) & (Original_Traget_Data==0) ).copy()
GoodGoodZero_Mean=hp(GoodGoodZeroPred.mean(),"Good Zero prediction Mean")

winratio=GoodTrueWinPred_Mean/(GoodTrueWinPred_Mean+GoodLossPred_Mean)
fiability=GoodTrueWinPred_Mean + GoodLossPred_Mean + GoodMissedDeal_Mean + GoodGoodZero_Mean
if( fiability == 100):print("good fiability")
else: print(f"check the fiability {fiability}")
print(f"========= Win Ratio:{winratio*100} ====================")
